## Main notebook to run and test the model

### 1. Setup & Imports

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd
import optuna
from src.config import Config
from src.utils import set_seed, plot_loss_curves, plot_results_publication
from src.data_loader import load_and_process_data
from src.model import DynamicMLP
from src.train import train_one_epoch, validate
from src.optimization import objective
from src.test import evaluate_model
from src.data_loader import load_test_data
from src.utils import EarlyStopping
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.tensorboard import SummaryWriter
from src.utils import save_artifacts

# Check device
device = Config.DEVICE
id_robot = 1 # 0 = 3 joint, 1 = 4 joint, 2 = 6 joint
writer = SummaryWriter(f"runs/{Config.ROBOT_CHOICE[id_robot]}_experiment")
print(f"Using device: {device}")

### 2. Robot Configuration

In [ ]:
# Configuration
ROBOT_CHOICE = Config.ROBOT_CHOICE[id_robot]  # Choose the robot configuration here

if ROBOT_CHOICE == "Reacher3":
    # Train file
    CSV_PATHS = [os.path.join(Config.DATA_DIR, f"reacher3_train_{i}.csv") for i in [1,2]]
    # Test file
    TEST_CSV_PATHS = [os.path.join(Config.DATA_DIR, f"reacher3_test_{i}.csv") for i in [1,2]]
    
    INPUT_COLS = Config.INPUTS_REACHER3
    OUTPUT_COLS = Config.OUTPUTS_REACHER3

elif ROBOT_CHOICE == "Reacher4":
    CSV_PATHS = [os.path.join(Config.DATA_DIR, f"reacher4_train_{i}.csv") for i in [1,2]]
    TEST_CSV_PATHS = [os.path.join(Config.DATA_DIR, f"reacher4_test_{i}.csv") for i in [1,2]]
    
    INPUT_COLS = Config.INPUTS_REACHER4
    OUTPUT_COLS = Config.OUTPUTS_REACHER4

elif ROBOT_CHOICE == "Reacher6":
    CSV_PATHS = [os.path.join(Config.DATA_DIR, f"reacher6_train_{i}.csv") for i in [1,2]]
    TEST_CSV_PATHS = [os.path.join(Config.DATA_DIR, f"reacher6_test_{i}.csv") for i in [1,2]]
    
    INPUT_COLS = Config.INPUTS_REACHER6
    OUTPUT_COLS = Config.OUTPUTS_REACHER6

print(f"Robot: {ROBOT_CHOICE}")
print(f"Train Files: {CSV_PATHS}")
print(f"Test Files: {TEST_CSV_PATHS}")

### 3. Data Loading & Preprocessing

In [ ]:
seed = Config.SEEDS[2]
set_seed(seed) # Reproducibility

# Load data
train_dataset, val_dataset, x_scaler, y_scaler = load_and_process_data(
    CSV_PATHS, INPUT_COLS, OUTPUT_COLS, val_split=0.2
)

# Create dataLoaders
train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE, shuffle=False)

input_dim = len(INPUT_COLS)
output_dim = len(OUTPUT_COLS)

print(f"Train samples: {len(train_dataset)}")
print(f"Val samples: {len(val_dataset)}")

### 4. Hyperparameter Optimization (Optuna)
We search for the best architecture (Layers, Units, LR). Then save the study result of each runn to have a complete overview

In [ ]:
if  Config.RUN_OPTIMIZATION:
    def run_optuna_study():
        study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=seed))
        
        study.optimize(
            lambda trial: objective(trial, train_loader, val_loader, input_dim, output_dim, device), 
            n_trials=Config.NUM_TRIALS
        )
        return study

    study = run_optuna_study()

    print("Best trial:")
    trial = study.best_trial
    print(f"  Value: {trial.value}")
    print(f"  Params: {trial.params}")

    # save the study results
    study_df = study.trials_dataframe()
    study_csv_path = os.path.join(Config.DATA_DIR, f"optuna_study_{ROBOT_CHOICE}.csv")
    study_df.to_csv(study_csv_path, index=False)
else:
    print("Skipping hyperparameter optimization as per configuration.")

### 5. Training the Best Model
Retrain the model with the best parameters found by Optuna.

In [ ]:
# Load params
if Config.LOAD_FROM_OPTUNA:
    all_study = pd.read_csv(os.path.join(Config.DATA_DIR, f"optuna_study_{ROBOT_CHOICE}.csv"))
    best_trial_index = all_study['value'].idxmin()
    best_row = all_study.iloc[best_trial_index]

    # Parse params
    n_layers = int(best_row['params_n_layers'])
    hidden_layers = [int(best_row[f'params_hidden_units_layer_{i}']) for i in range(n_layers)]
    dropout = float(best_row['params_dropout'])
    lr = float(best_row['params_lr'])
    activation = best_row['params_activation']
    raw_res = best_row.get('params_use_residual', False) 
    use_residual = str(raw_res) == "True" or raw_res is True
else:
    # Get best params from Config
    best_params = Config.best_hyperparameters(ROBOT_CHOICE)
    n_layers = best_params['num_layers']
    hidden_layers = best_params['hidden_units']
    dropout = best_params['dropout']
    lr = best_params['learning_rate']
    activation = best_params['activation']
    use_residual = best_params['use_residuals']


print(f"Best Config: Res={use_residual}, Act={activation}, Layers={hidden_layers}")

# Build model
model = DynamicMLP(input_dim, output_dim, hidden_layers, dropout, activation=activation, use_residual=use_residual).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss()

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
save_path = os.path.join(Config.MODELS_DIR, f"{ROBOT_CHOICE}_best.pth")
early_stopping = EarlyStopping(patience=15, min_delta=1e-6, path=save_path)

# Training loop
train_losses = []
val_losses = []

print(f"Starting robust training (Max Epochs: {Config.EPOCHS})...")

for epoch in range(Config.EPOCHS):
    t_loss = train_one_epoch(model, train_loader, optimizer, criterion, device, epoch+1, writer)
    v_loss, v_r2 = validate(model, val_loader, criterion, device, epoch+1, writer)
    
    train_losses.append(t_loss)
    val_losses.append(v_loss)
    
    scheduler.step(v_loss)
    early_stopping(v_loss, model)
    
    if early_stopping.early_stop:
        print(f"Early stopping triggered at epoch {epoch+1}")
        break

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1} | Train: {t_loss:.5f} | Val: {v_loss:.5f} | R2: {v_r2:.4f}")

# Finalize
print("Loading best model weights...")
model.load_state_dict(torch.load(save_path))

# Plot
plot_save_path = os.path.join(Config.PLOTS_DIR, f"{ROBOT_CHOICE}_loss.png")
plot_loss_curves(train_losses, val_losses, save_path=plot_save_path)

# Save artifacts
save_artifacts(model, x_scaler, y_scaler, 
               save_dir=Config.MODELS_DIR, 
               model_name=f"{ROBOT_CHOICE}_final")

### 6. Final Evaluation & Visualization
Compute metrics on the validation set in the ORIGINAL scale (Radians).

In [ ]:
# Load test data
test_dataset = load_test_data(TEST_CSV_PATHS, INPUT_COLS, OUTPUT_COLS, x_scaler, y_scaler)
test_loader = DataLoader(test_dataset, batch_size=Config.BATCH_SIZE, shuffle=False)

print(f"Loaded Test Set: {len(test_dataset)} samples")

# Reconstruct model
if Config.LOAD_FROM_OPTUNA:
    all_study = pd.read_csv(os.path.join(Config.DATA_DIR, f"optuna_study_{ROBOT_CHOICE}.csv"))
    best_trial_index = all_study['value'].idxmin()
    best_row = all_study.iloc[best_trial_index]

    # Parse params
    n_layers = int(best_row['params_n_layers'])
    hidden_layers = [int(best_row[f'params_hidden_units_layer_{i}']) for i in range(n_layers)]
    dropout = float(best_row['params_dropout'])
    lr = float(best_row['params_lr'])
    activation = best_row['params_activation']
    raw_res = best_row.get('params_use_residual', False) 
    use_residual = str(raw_res) == "True" or raw_res is True
else:
    # Get best params from Config
    best_params = Config.best_hyperparameters(ROBOT_CHOICE)
    n_layers = best_params['num_layers']
    hidden_layers = best_params['hidden_units']
    dropout = best_params['dropout']
    lr = best_params['learning_rate']
    activation = best_params['activation']
    use_residual = best_params['use_residuals']

model = DynamicMLP(input_dim, output_dim, hidden_layers, dropout, activation=activation, use_residual=use_residual).to(device)
# Load Weights
weights_path = os.path.join(Config.MODELS_DIR, f"{ROBOT_CHOICE}_best.pth")
model.load_state_dict(torch.load(weights_path))
print(f"Model loaded from {weights_path}")

# Evaluate on test set
metrics, y_true, y_pred = evaluate_model(model, test_loader, y_scaler, device)

print(f"--- Final Test Results ({ROBOT_CHOICE}) ---")
print(f"R2 Score: {metrics['r2']:.4f}")
print(f"MSE (Radians^2): {metrics['mse']:.6e}")

# Plots
num_joints = output_dim
for i in range(num_joints):
    plot_results_publication(y_true, y_pred, joint_idx=i, 
                             robot_name=ROBOT_CHOICE, 
                             save_dir=Config.PLOTS_DIR)

### 7. Cleanup

In [ ]:
import gc
import torch

def cleanup_memory():
    """
    Forcefully frees up RAM and VRAM.
    """
    gc.collect()
    
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
        
    print("Memory cleaned.")
    if torch.cuda.is_available():
        print(f"GPU Allocated: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
        print(f"GPU Reserved:  {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

vars_to_delete = ['model', 'optimizer', 'scheduler', 'train_loader', 'val_loader', 'test_loader', 'train_dataset', 'val_dataset', 'study']

for var in vars_to_delete:
    if var in globals():
        del globals()[var]

cleanup_memory()
writer.flush()
writer.close()